In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import date
import time
%matplotlib inline
import numpy as np
from numpy import nan

In [2]:
data_folder = "../Data-Analytics-DRMIS-SAP-Production-DATA/"
df = pd.read_csv(data_folder + "DRMIS work data.csv", encoding = 'ISO-8859-1')
df = df.fillna('nil')
def str2date(str):
    try:
        day = int(str[0:2])
        month = int(str[3:5])
        year = int(str[6:10])
        datee = date(year, month, day)
        return(datee)
    except:
        pass

days_WIP = [0]*len(df)
act_rel_date = list(df['Actual release date'])
bsc_fin_date = list(df['Basic finish date'])

for i in range(len(df)):
    try:
        days_WIP[i] = int((str2date(bsc_fin_date[i]) - str2date(act_rel_date[i])).days)
    except:
        days_WIP[i] = 0
        
    if days_WIP[i] <=0:
        days_WIP[i] = 0

df['days WIP'] = days_WIP 
df.drop(['Basic start date', 'Actual start date', 'Scheduled finish', 'Scheduled start', 'Actual Finish Time'], axis = 1, inplace = True)
vehicle_df = df[df['Tail Number / Identifier']!='nil'].copy()

In [ ]:
vehicle_df.to_csv(data_folder + "vehicle_df.csv")

In [3]:
vehicle_df.head(3)

,Order,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Basic finish date,days WIP
7,1102603947,21293-TRUCK CARRYALL INSP-12 MONTH,448.48,4.0,nil,0.0,2017-21293,CRTD,2.09598e+07,L27,AG-I---2,EV0A80,102,nil,27/07/2020,0
8,1102603946,20031-TRK CARGO TO 3 1/2 PERIODIC INSPEC,448.48,4.0,nil,0.0,2017-20031,CRTD,2.09386e+07,L27,AG-I---2,EV0B16,102,nil,27/07/2020,0
9,1102603468,16650-TRK CARGO TO 3 1/2 T INSP-12 MONTH,448.48,4.0,nil,0.0,2015-16650,CRTD,2.08952e+07,L27,AG-I---2,EV0B16,102,nil,25/07/2020,0


In [4]:
unique_CFRs = list(vehicle_df['Tail Number / Identifier'].unique())
unique_veh_type = list(vehicle_df['Vehicle Type'].unique())

unique_days_WIP = [0]*len(unique_CFRs)
veh_type = [0]*len(unique_CFRs)

CFRs = list(vehicle_df['Tail Number / Identifier'])
days_WIP = list(vehicle_df['days WIP'])

for a in range(len(unique_CFRs)):
    for b in range(len(CFRs)):
        if CFRs[b]==unique_CFRs[a]:
            unique_days_WIP[a]+=days_WIP[b]
        
        
data = {'unique CFRs': unique_CFRs, 'unique days WIP':unique_days_WIP, 'vehicle type':veh_type}
byCFR = pd.DataFrame(data)

for i in range(len(byCFR)):
    byCFR['vehicle type'][i] = list(df[df['Tail Number / Identifier'] == byCFR['unique CFRs'][i]]['Vehicle Type'])[0]

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [72]:
unique_veh_type = list(vehicle_df['Vehicle Type'].unique())

days_WIP_per = [0.0]*len(unique_veh_type)
days_WIP_per_veh = [0.0]*len(unique_veh_type)
num_veh = [0.0]*len(unique_veh_type)

hours_total = [0.0]*len(unique_veh_type)
hours_per = [0.0]*len(unique_veh_type)

num_WO = [0.0]*len(unique_veh_type)
num_WO_per_veh = [0.0]*len(unique_veh_type)

avg_hours_per_WO = [0.1]*len(unique_veh_type)

for a in range(len(days_WIP_per)):
    for b in range(len(unique_days_WIP)):
        if veh_type[b]==unique_veh_type[a]:
            days_WIP_per[a]+=unique_days_WIP[b]
            
databytype = {'unique vehicle type':unique_veh_type,'num veh': num_veh,
              'days WIP total':days_WIP_per, 'days WIP per veh':days_WIP_per_veh,
              'hours worked':hours_total, 'hours per vehicle':hours_per,
              'Work Orders':num_WO, 'Work Orders per vehicle':num_WO_per_veh,
             'avg hours per work order':avg_hours_per_WO}
bytype = pd.DataFrame(databytype)       


unique_days_WIP = list(byCFR['unique days WIP'])
veh_type = list(byCFR['vehicle type'])

for i in range(len(bytype)):
    x = bytype['unique vehicle type'][i]
    bytype['num veh'][i] = byCFR['vehicle type'].value_counts()[x]

for i in range(len(bytype)):
    bytype['days WIP per veh'][i] = bytype['days WIP total'][i]/bytype['num veh'][i]
    
    
for a in range(len(bytype)):
    type = bytype['unique vehicle type'][a]
    bytype['hours worked'][a] = sum(vehicle_df[vehicle_df['Vehicle Type']==type]['PM Orders actual hours'])
    bytype['hours per vehicle'][a] = sum(vehicle_df[vehicle_df['Vehicle Type']==type]['PM Orders actual hours'])/bytype['num veh'][a]
    
for i in range(len(bytype)):
    type = bytype['unique vehicle type'][i]
    bytype['Work Orders'][i] = len(vehicle_df[vehicle_df['Vehicle Type']==type])
    bytype['Work Orders per vehicle'][i] = bytype['Work Orders'][i]/bytype['num veh'][i]
    
for i in range(len(bytype)):
    bytype['avg hours per work order'][i] = bytype['hours per vehicle'][i]/bytype['Work Orders per vehicle'][i]

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\olfoa\a

In [73]:
bytype.sort_values(by = 'avg hours per work order', ascending = False)

,unique vehicle type,num veh,days WIP total,days WIP per veh,hours worked,hours per vehicle,Work Orders,Work Orders per vehicle,avg hours per work order
45,EV0B30,1.0,236.0,236.000000,224.00,224.000000,11.0,11.000000,20.363636
16,EV0B27,1.0,502.0,502.000000,351.00,351.000000,18.0,18.000000,19.500000
52,EV0D75,5.0,3964.0,792.800000,2277.50,455.500000,128.0,25.600000,17.792969
38,EV0D65,5.0,4386.0,877.200000,2505.50,501.100000,162.0,32.400000,15.466049
36,EV0E65,1.0,184.0,184.000000,146.00,146.000000,10.0,10.000000,14.600000
...,...,...,...,...,...,...,...,...,...
44,EV0C96,4.0,336.0,84.000000,25.50,6.375000,15.0,3.750000,1.700000
55,EV0F65,2.0,163.0,81.500000,24.00,12.000000,15.0,7.500000,1.600000
20,EV0B59,13.0,3530.0,271.538462,116.75,8.980769,146.0,11.230769,0.799658
41,EV0B96,9.0,5295.0,588.333333,80.00,8.888889,168.0,18.666667,0.476190


In [74]:
bytype_norm = bytype.copy()
#bytype['hours worked'].mean()
#bytype['hours worked'].std()

for col in list(bytype_norm.columns)[2:-1]:
    for i in range(len(bytype_norm)):
        bytype_norm[col][i] = bytype[col][i]/bytype[col].mean()

bytype_norm['burden factor'] = [0.0]*len(bytype_norm)

for i in range(len(bytype_norm)):
    work_hours_factor = bytype_norm['hours worked'][i]/sum(bytype_norm['hours worked'])
    veh_num_factor = bytype_norm['num veh'][i]/(sum(bytype_norm['num veh']))
    bytype_norm['burden factor'][i] = work_hours_factor / veh_num_factor

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [75]:
bytype_norm.sort_values(by = 'burden factor', ascending = False)[0:10].reset_index(drop = True)

,unique vehicle type,num veh,days WIP total,days WIP per veh,hours worked,hours per vehicle,Work Orders,Work Orders per vehicle,avg hours per work order,burden factor
0,EV0B26,4.0,1.641021,2.985809,4.276342,5.793832,1.679893,3.212796,13.150718,9.791465
1,EV0B29,2.0,0.618294,2.249950,1.621229,4.393067,0.643021,2.459557,13.025000,7.424200
2,EV0D65,5.0,1.702346,2.477911,3.898263,4.225271,1.302118,1.992241,15.466049,7.140627
3,EV0D75,5.0,1.538554,2.239498,3.543521,3.840772,1.028834,1.574116,17.792969,6.490831
4,EV0B46,3.0,1.265698,3.070553,2.032760,3.672132,0.811814,2.070127,12.935644,6.205833
5,EV0B40,11.0,7.134639,4.720490,7.428559,3.659867,6.952666,4.835265,5.519653,6.185106
6,EV0D80,5.0,1.555244,2.263791,2.814591,3.050694,1.366420,2.090623,10.641176,5.155615
7,EV0B27,1.0,0.194842,1.418047,0.546115,2.959629,0.144680,1.106800,19.500000,5.001716
8,EV0A30,2.0,0.774323,2.817734,1.019492,2.762531,0.618908,2.367323,8.509740,4.668625
9,EV0B44,6.0,1.651501,2.003251,2.305817,2.082702,1.551289,1.977893,7.678756,3.519726


In [81]:
bytype_sdev= bytype.copy()
#bytype['hours worked'].mean()
#bytype['hours worked'].std()

for col in list(bytype_sdev.columns)[2:-1]:
    for i in range(len(bytype_sdev)):
        bytype_sdev[col][i] = (bytype[col][i] - bytype[col].mean())/bytype[col].std()

bytype_sdev['divergence factor'] = [0.0]*len(bytype_norm)

for i in range(len(bytype_sdev)):
    for col in list(bytype_sdev.columns)[2:-1]:
        bytype_sdev['divergence factor'][i] += abs(bytype_sdev[col][i])
        
    


c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [84]:
bytype_sdev.sort_values(by = 'divergence factor', ascending = False)[0:10].reset_index(drop = True)

,unique vehicle type,num veh,days WIP total,days WIP per veh,hours worked,hours per vehicle,Work Orders,Work Orders per vehicle,avg hours per work order,divergence factor
0,EV0B40,11.0,3.846365,4.015790,4.290198,2.040296,3.344103,4.403204,5.519653,27.459611
1,EV0B26,4.0,0.401915,2.143425,2.186517,3.677190,0.381952,2.540474,13.150718,24.482191
2,EV0D75,5.0,0.337669,1.337879,1.697458,2.179062,0.016198,0.659133,17.792969,24.020369
3,EV0B30,1.0,-0.569560,-0.359807,-0.434777,0.681743,-0.512112,-0.371545,20.363636,23.293179
4,EV0D65,5.0,0.440365,1.595214,1.934200,2.473999,0.169725,1.139175,15.466049,23.218728
5,EV0B27,1.0,-0.504827,0.451228,-0.302907,1.503167,-0.480504,0.122616,19.500000,22.865249
6,EV0B29,2.0,-0.239326,1.349160,0.414587,2.602710,-0.200544,1.675693,13.025000,19.507020
7,EV0B46,3.0,0.166590,2.234895,0.689228,2.049704,-0.105719,1.228595,12.935644,19.410376
8,EV0B16,109.0,5.002060,-0.432339,3.382428,-0.535632,6.035326,-0.201859,2.669576,18.259219
9,EV0B54,2.0,-0.618717,-1.027538,-0.610776,-0.590817,-0.543720,-1.006895,13.625000,18.023463


# Analyze work orders (hours worked and days open) per vehicle type

days WIP = for all work orders against all vehicle of this type, how many working days have they been open

hours worked = hours worked on all veicles of that type

In [204]:
bytype.sort_values(by='hours worked', ascending = False).to_csv('bytype.csv')

In [191]:
sum(bytype['hours worked'])

40474

In [14]:
vehicle_df.head(3)

,Order,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Basic finish date,days WIP
7,1102603947,21293-TRUCK CARRYALL INSP-12 MONTH,448.48,4.0,nil,0.0,2017-21293,CRTD,2.09598e+07,L27,AG-I---2,EV0A80,102,nil,27/07/2020,0
8,1102603946,20031-TRK CARGO TO 3 1/2 PERIODIC INSPEC,448.48,4.0,nil,0.0,2017-20031,CRTD,2.09386e+07,L27,AG-I---2,EV0B16,102,nil,27/07/2020,0
9,1102603468,16650-TRK CARGO TO 3 1/2 T INSP-12 MONTH,448.48,4.0,nil,0.0,2015-16650,CRTD,2.08952e+07,L27,AG-I---2,EV0B16,102,nil,25/07/2020,0


In [163]:
#vehicle_df = vehicle_df.reset_index(drop = True)
work_type = [0]*len(vehicle_df)
vehicle_df['work type'] = work_type

def get_work_type(str):
    insp_list = ['inspection', 'insp', 'annual', '12', 'period', 'month']
    work_list = ['after', 'post']
    
    insp = any(word in str.lower() for word in insp_list)
    work = any(word in str.lower() for word in work_list)
    
    if insp == True and work == False:
        return('inspection')
    
    else:
        return('corrective')
    
for i in range(len(vehicle_df)):
    vehicle_df['work type'][i] = get_work_type(vehicle_df['Description'][i])

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


# it appears that MaintActivityType of L27 is inspection, and L01 is corrective work

In [180]:
vehicle_df[vehicle_df['work type']=='inspection'].head()

,Order,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Basic finish date,days WIP,work type
0,1102603947,21293-TRUCK CARRYALL INSP-12 MONTH,448.48,4.0,nil,0.0,2017-21293,CRTD,2.09598e+07,L27,AG-I---2,EV0A80,102,nil,27/07/2020,0,inspection
1,1102603946,20031-TRK CARGO TO 3 1/2 PERIODIC INSPEC,448.48,4.0,nil,0.0,2017-20031,CRTD,2.09386e+07,L27,AG-I---2,EV0B16,102,nil,27/07/2020,0,inspection
2,1102603468,16650-TRK CARGO TO 3 1/2 T INSP-12 MONTH,448.48,4.0,nil,0.0,2015-16650,CRTD,2.08952e+07,L27,AG-I---2,EV0B16,102,nil,25/07/2020,0,inspection
3,1102602653,22711-FORKLIFT ICED RT INSP-12 MONTH,448.48,4.0,nil,0.0,2017-22711,CRTD,2.10198e+07,L27,AG-S--V2,EV0F81,102,nil,23/07/2020,0,inspection
4,1102602646,18070-TRK CARGO TO 3 1/2 T INSP-12 MONTH,448.48,4.0,nil,0.0,2016-18070,CRTD,2.09161e+07,L27,AG-I---2,EV0B16,102,nil,23/07/2020,0,inspection


In [179]:
vehicle_df[vehicle_df['work type']=='corrective'].head()

,Order,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Basic finish date,days WIP,work type
15,1102603965,Mobile Crane Certification [24268],"1,345.44",12.0,SERV,0.0,2018-24268,ACTL,2.10848e+07,L27,AG-LS-V2,EV0C16,102,23/06/2020,14/07/2020,21,corrective
28,1102604277,24772- Replace windshield,112.12,1.0,nil,0.0,2019-24772,TAPP,2.11428e+07,L01,AG-S--V2,EV0E80,102,24/06/2020,24/06/2020,0,corrective
30,1102604872,20233-Repairs after,0,0.0,GRND,0.0,2017-20233,ACTL,2.09504e+07,nil,AG-LL-V2,EV0B16,102,24/06/2020,24/06/2020,0,corrective
31,1102593451,*19995 SAFETY RECALL N182165790,0,0.0,SERV,1.0,2017-19995,OAPP,2.09382e+07,L01,AG-LL-V2,EV0B34,4026,28/05/2020,24/06/2020,27,corrective
32,1102604818,*16767 HYDRAULIC LEAK,0,0.0,nil,0.0,2014-16767,CRTD,2.08976e+07,L01,AG-S--V2,EV0F63,102,nil,nil,0,corrective


In [27]:
work_centers = list(vehicle_df['Main work center'].unique())
num_workorder_workcenter = vehicle_df['Main work center'].value_counts()
days_WIP_work_center = [0.0]*len(work_centers)
days_WIP_work_order_work_center = [0.0]*len(work_centers)
hours_per_work_center = [0.0]*len(work_centers)
hours_per_work_order_work_center = [0.0]*len(work_centers)

databyworkcenter = {'work center':work_centers,'num work order': num_workorder_workcenter,
                   'days WIP':days_WIP_work_center, 'days WIP per work order':days_WIP_work_order_work_center,
                   'hours':hours_per_work_center, 'hours per work order':hours_per_work_order_work_center}
byworkcenter = pd.DataFrame(databyworkcenter).reset_index(drop=True)

for i in range(len(byworkcenter)):
    work_center = byworkcenter['work center'][i]
    byworkcenter['days WIP'][i] = sum(vehicle_df[vehicle_df['Main work center']==work_center]['days WIP'])
    byworkcenter['days WIP per work order'][i] = byworkcenter['days WIP'][i]/byworkcenter['num work order'][i]
    byworkcenter['hours'][i] = sum(vehicle_df[vehicle_df['Main work center']==work_center]['PM Orders actual hours'])
    byworkcenter['hours per work order'][i] = byworkcenter['hours'][i]/byworkcenter['num work order'][i]
byworkcenter

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

,work center,num work order,days WIP,days WIP per work order,hours,hours per work order
0,AG-I---2,1653,21991.0,13.303690,3644.00,2.204477
1,AG-S--V2,1468,33654.0,22.925068,3996.25,2.722241
2,AG-LS-V2,1330,38223.0,28.739098,16613.00,12.490977
3,AG-LL-V2,1169,21791.0,18.640719,5048.75,4.318862
4,AG-LR-V2,906,19269.0,21.268212,4973.00,5.488962
5,AG-O--M2,682,2342.0,3.434018,905.50,1.327713
6,AG-LH-V2,321,13338.0,41.551402,4355.50,13.568536
7,AG-R--E2,210,182.0,0.866667,98.50,0.469048
8,AG-LB-V2,40,9518.0,237.950000,264.75,6.618750
9,AG-N---2,30,294.0,9.800000,184.50,6.150000


In [30]:
Maint_Activity_Type = list(vehicle_df['MaintActivityType'].unique())
num_workorder_MaintActivity = vehicle_df['MaintActivityType'].value_counts()
days_WIP_maint_activity = [0.0]*len(Maint_Activity_Type)
days_WIP_work_order_maint_activity = [0.0]*len(Maint_Activity_Type)
hours_per_maint_activity = [0.0]*len(Maint_Activity_Type)
hours_per_work_order_maint_activity = [0.0]*len(Maint_Activity_Type)


databymaintactivity = {'Maint Activity Type': Maint_Activity_Type, 'num work order':num_workorder_MaintActivity,
                      'days WIP':days_WIP_maint_activity, 'days WIP per work order':days_WIP_work_order_maint_activity,
                      'hours':hours_per_maint_activity, 'hours per work order':hours_per_work_order_maint_activity}
bymaintactivity = pd.DataFrame(databymaintactivity).reset_index(drop=True)   

for i in range(len(bymaintactivity)):
    maint_activity = bymaintactivity['Maint Activity Type'][i]
    bymaintactivity['days WIP'][i] = sum(vehicle_df[vehicle_df['MaintActivityType']==maint_activity]['days WIP'])
    bymaintactivity['days WIP per work order'][i] = bymaintactivity['days WIP'][i]/bymaintactivity['num work order'][i]
    bymaintactivity['hours'][i] = sum(vehicle_df[vehicle_df['MaintActivityType']==maint_activity]['PM Orders actual hours'])
    bymaintactivity['hours per work order'][i] = bymaintactivity['hours'][i]/bymaintactivity['num work order'][i]
bymaintactivity

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,Maint Activity Type,num work order,days WIP,days WIP per work order,hours,hours per work order
0,L27,4640,65725.0,14.164871,9752.00,2.101724
1,L01,2501,83459.0,33.370252,26309.25,10.519492
2,nil,475,7674.0,16.155789,3901.50,8.213684
3,L16,136,13.0,0.095588,1.50,0.011029
4,L28,56,4639.0,82.839286,249.00,4.446429
5,L02,14,0.0,0.000000,5.00,0.357143
6,L03,7,338.0,48.285714,233.50,33.357143
7,L17,4,277.0,69.250000,24.75,6.187500
8,L06,3,0.0,0.000000,0.00,0.000000
9,L05,1,191.0,191.000000,9.00,9.000000


In [11]:
df.sort_values(by = 'Actual release date', ascending = False)

,Order,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Basic finish date,days WIP
420,1102582384,23369-TLR COMMERCIAL INSP-12 MONTH,112.12,1.0,nil,0.0,2019-23369,CRTD,2.10299e+07,L27,AG-S--V2,EV0017,102,nil,07/05/2020,0
100,1102604107,*23281 OIL CHANGE REQUIRED,0,0.0,nil,0.0,2018-23281,ACTL,2.10263e+07,L01,AG-LL-V2,EV0B34,102,nil,nil,0
376,1102584218,82981- TRAILER MISC INSP-12 MONTH,336.36,3.0,nil,0.0,2008-82981,CRTD,2.01113e+07,L27,AG-S--V2,EV0017,102,nil,17/05/2020,0
377,1102584217,82943- TRAILER MISC INSP-12 MONTH,336.36,3.0,nil,0.0,2008-82943,CRTD,2.01113e+07,L27,AG-S--V2,EV0017,102,nil,17/05/2020,0
378,1102584167,22973-TRK CARGO TO 3 1/2 T INSP-12 MONTH,448.48,4.0,GRND,0.0,2018-22973,CRTD,2.10226e+07,L27,AG-I---2,EV0B16,102,nil,16/05/2020,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12596,1101873564,Ref - B-620 Inspection 1 Year,0,0.0,SERV,0.0,2015-13749,OAPP,2.08461e+07,L01,AG-LR-V2,EV0B40,102,01/02/2016,01/02/2016,0
12595,1101873565,Ref - B-620 Inspection 1 Year,0,0.0,SERV,0.0,2015-13750,OAPP,2.08461e+07,L01,AG-LR-V2,EV0B40,102,01/02/2016,01/02/2016,0
12594,1101873567,Ref - B-620 Inspection 1 Year,0,0.0,SERV,0.0,2015-13751,OAPP,2.08461e+07,L01,AG-LR-V2,EV0B40,102,01/02/2016,01/02/2016,0
12300,1101872937,Perform twelve month inspection IAW appl,106.2,1.0,nil,4.0,nil,OAPP,2.05317e+07,L27,AG-N---2,nil,0,01/02/2016,03/03/2016,31


In [32]:
vehicle_df.to_csv('vehicle_df.csv')

In [43]:
for i in range(0,30):
    print(str(i) + ' days WIP ---> ' + str(len(vehicle_df[vehicle_df['days WIP']>i])) + ' work orders')


0 days WIP ---> 3848 work orders
1 days WIP ---> 3621 work orders
2 days WIP ---> 3521 work orders
3 days WIP ---> 3431 work orders
4 days WIP ---> 3363 work orders
5 days WIP ---> 3292 work orders
6 days WIP ---> 3191 work orders
7 days WIP ---> 3076 work orders
8 days WIP ---> 3003 work orders
9 days WIP ---> 2957 work orders
10 days WIP ---> 2916 work orders
11 days WIP ---> 2865 work orders
12 days WIP ---> 2816 work orders
13 days WIP ---> 2758 work orders
14 days WIP ---> 2683 work orders
15 days WIP ---> 2634 work orders
16 days WIP ---> 2596 work orders
17 days WIP ---> 2569 work orders
18 days WIP ---> 2540 work orders
19 days WIP ---> 2515 work orders
20 days WIP ---> 2479 work orders
21 days WIP ---> 2441 work orders
22 days WIP ---> 2409 work orders
23 days WIP ---> 2385 work orders
24 days WIP ---> 2350 work orders
25 days WIP ---> 2315 work orders
26 days WIP ---> 2292 work orders
27 days WIP ---> 2237 work orders
28 days WIP ---> 2179 work orders
29 days WIP ---> 2131 wo

In [35]:
len(vehicle_df)

7838

In [44]:
for i in range(0,30):
    print(str(i) + ' hours ---> ' + str(len(vehicle_df[vehicle_df['PM Orders actual hours']>i])) + ' work orders')

0 hours ---> 6871 work orders
1 hours ---> 5328 work orders
2 hours ---> 4154 work orders
3 hours ---> 3289 work orders
4 hours ---> 2282 work orders
5 hours ---> 1703 work orders
6 hours ---> 1426 work orders
7 hours ---> 1245 work orders
8 hours ---> 1064 work orders
9 hours ---> 939 work orders
10 hours ---> 829 work orders
11 hours ---> 748 work orders
12 hours ---> 675 work orders
13 hours ---> 617 work orders
14 hours ---> 551 work orders
15 hours ---> 506 work orders
16 hours ---> 470 work orders
17 hours ---> 441 work orders
18 hours ---> 412 work orders
19 hours ---> 383 work orders
20 hours ---> 356 work orders
21 hours ---> 333 work orders
22 hours ---> 309 work orders
23 hours ---> 286 work orders
24 hours ---> 264 work orders
25 hours ---> 245 work orders
26 hours ---> 233 work orders
27 hours ---> 224 work orders
28 hours ---> 212 work orders
29 hours ---> 203 work orders


7838

In [4]:
import os
import pandas as pd

In [5]:

data_folder = "../Data-Analytics-DRMIS-SAP-Production-DATA/DRMIS work data.csv"

df2 = pd.read_csv("../Data-Analytics-DRMIS-SAP-Production-DATA/DRMIS work data.csv", encoding = 'ISO-8859-1' )



,Order,Basic start date,Description,Total planned costs,PM Orders planned hours,Technical Status,PM Orders actual hours,Tail Number / Identifier,User Status,Equipment,MaintActivityType,Main work center,Vehicle Type,Utilizer,Actual release date,Actual start date,Actual Finish Time,Basic finish date,Scheduled finish,Scheduled start
0,1102603954,21/09/2020,CFCU2004074-CARGO SEA CAN INSP-30 MTH,224.24,2.0,NaN,0.0,NaN,CANC,20676888.0,L27,AG-O--M2,NaN,0,25/06/2020,NaN,12:00:00 AM,21/09/2020,21/09/2020,21/09/2020
1,1102603952,21/09/2020,CFCU1101418-CARGO SEA CAN INSP-30 MTH,224.24,2.0,NaN,0.0,NaN,CANC,20597759.0,L27,AG-O--M2,NaN,0,25/06/2020,NaN,12:00:00 AM,21/09/2020,21/09/2020,21/09/2020
2,1102603953,21/09/2020,CFCU2000695-CARGO SEA CAN INSP-30 MTH,224.24,2.0,NaN,0.0,NaN,CANC,20570294.0,L27,AG-O--M2,NaN,0,25/06/2020,NaN,12:00:00 AM,21/09/2020,21/09/2020,21/09/2020
3,1102603937,02/09/2020,2017-1 Perform Twenty four month inspect,224.24,2.0,NaN,0.0,NaN,ACTL,20531873.0,L27,AG-N---2,NaN,0,23/06/2020,NaN,12:00:00 AM,02/09/2020,02/09/2020,02/09/2020
4,1102605100,31/07/2020,Perform 12 month inspection iaw applicab,224.24,2.0,NaN,0.0,NaN,ACTL,20531664.0,L27,AG-N---2,NaN,0,25/06/2020,NaN,12:00:00 AM,31/07/2020,31/07/2020,31/07/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13172,1101730708,23/04/2015,SPV - Repairs After Inspection,"2,059.76",0.0,SERV,30.0,2005-65251,OAPP,20110745.0,L01,AG-LS-V2,EV0D65,102,30/03/2015,23/04/2015,12:00:00 AM,09/07/2015,09/07/2015,23/04/2015
13173,1101736229,17/04/2015,Repairs After Inspection,0,0.0,SERV,25.0,NaN,OAPP,20118886.0,L01,AG-LS-V2,NaN,0,14/04/2015,17/04/2015,12:00:00 AM,12/04/2016,12/04/2016,17/04/2015
13174,1101544815,05/03/2014,Completed on Another WiO,0,0.0,NaN,0.0,NaN,OAPP,20811378.0,L01,AG-B---2,NaN,0,05/03/2014,NaN,12:00:00 AM,05/03/2014,05/03/2014,05/03/2014
13175,1101460908,16/12/2013,WCE/A&G-Replace Temp Gauges 010218,0,0.0,NaN,0.5,NaN,OAPP,NaN,L01,AG-S--V2,NaN,0,12/09/2013,16/12/2013,12:00:00 AM,26/07/2016,26/07/2016,16/12/2013
